In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)
import matplotlib.pyplot as plt


In [2]:
# Import csv into a DataFrame.
weather_data_df = pd.read_csv("weather_data_challenge/WeatherPy_challenge.csv")
weather_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Recent Rain,Recent Snow,Country,Date
0,0,Barrow,71.29,-156.79,-14.80,75,1,6.93,clear sky,0.0,0.0,US,2020-02-23 02:28:58
1,1,Esperance,-33.87,121.90,78.80,57,40,5.82,scattered clouds,0.0,0.0,AU,2020-02-23 02:35:43
2,2,Rikitea,-23.12,-134.97,79.86,66,12,6.15,few clouds,0.0,0.0,PF,2020-02-23 02:28:59
3,3,Kulhudhuffushi,6.62,73.07,83.57,76,100,19.51,overcast clouds,0.0,0.0,MV,2020-02-23 02:38:04
4,4,Punta Arenas,-53.15,-70.92,51.80,66,20,17.22,few clouds,0.0,0.0,CL,2020-02-23 02:28:57


In [3]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain = str(input("Do you want it to be raining? (yes/no) "))
snow = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [4]:
# Filter the dataset to find the cities that fit the criteria.
if rain == "yes":
    rain_condition = weather_data_df["Recent Rain"] > 0
else:
    rain_condition = weather_data_df["Recent Rain"] == 0
        
if snow == "yes":
    snow_condition = weather_data_df["Recent Snow"] > 0
else:
    snow_condition = weather_data_df["Recent Snow"] == 0        
        
preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & 
                                       (weather_data_df["Max Temp"] >= min_temp) & rain_condition & snow_condition]
if len(preferred_cities_df) == 0:
    print ("Unfortunately we couldn't find the places that match your criteria. Please enter another values.")
preferred_cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Recent Rain,Recent Snow,Country,Date
1,1,Esperance,-33.87,121.90,78.80,57,40,5.82,scattered clouds,0.0,0.0,AU,2020-02-23 02:35:43
2,2,Rikitea,-23.12,-134.97,79.86,66,12,6.15,few clouds,0.0,0.0,PF,2020-02-23 02:28:59
3,3,Kulhudhuffushi,6.62,73.07,83.57,76,100,19.51,overcast clouds,0.0,0.0,MV,2020-02-23 02:38:04
7,7,Kapaa,22.08,-159.32,80.60,69,20,8.05,few clouds,0.0,0.0,US,2020-02-23 02:34:39
8,8,Nieuw Amsterdam,5.88,-55.08,78.01,94,40,3.36,scattered clouds,0.0,0.0,SR,2020-02-23 02:38:06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,529,Emerald,-23.53,148.17,82.40,65,90,18.34,overcast clouds,0.0,0.0,AU,2020-02-23 02:40:00
532,532,Isangel,-19.55,169.27,84.20,79,20,16.04,few clouds,0.0,0.0,VU,2020-02-23 02:35:05
533,533,Paracuru,-3.41,-39.03,78.80,94,75,4.70,light rain,0.0,0.0,BR,2020-02-23 02:40:01
536,536,Satun,6.83,100.00,86.00,70,40,14.99,scattered clouds,0.0,0.0,TH,2020-02-23 02:40:02


In [5]:
#drop rows with na
preferred_cities_df=preferred_cities_df.dropna()

In [6]:
len(preferred_cities_df)

134

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng", "Current Weather"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Lat,Lng,Current Weather,Hotel Name
1,Esperance,AU,78.80,-33.87,121.90,scattered clouds,
2,Rikitea,PF,79.86,-23.12,-134.97,few clouds,
3,Kulhudhuffushi,MV,83.57,6.62,73.07,overcast clouds,
7,Kapaa,US,80.60,22.08,-159.32,few clouds,
8,Nieuw Amsterdam,SR,78.01,5.88,-55.08,scattered clouds,
...,...,...,...,...,...,...,...
529,Emerald,AU,82.40,-23.53,148.17,overcast clouds,
532,Isangel,VU,84.20,-19.55,169.27,few clouds,
533,Paracuru,BR,78.80,-3.41,-39.03,light rain,
536,Satun,TH,86.00,6.83,100.00,scattered clouds,


In [8]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [9]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
hotel_df.head(10)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Lat,Lng,Current Weather,Hotel Name
1,Esperance,AU,78.80,-33.87,121.90,scattered clouds,Hospitality Esperance
2,Rikitea,PF,79.86,-23.12,-134.97,few clouds,Pension Maro'i
3,Kulhudhuffushi,MV,83.57,6.62,73.07,overcast clouds,Haajy Guest House 1
7,Kapaa,US,80.60,22.08,-159.32,few clouds,Sheraton Kauai Resort at Coconut Beach
8,Nieuw Amsterdam,SR,78.01,5.88,-55.08,scattered clouds,Hotel Stardust
11,Vaini,TO,84.20,-21.20,-175.20,scattered clouds,Keleti Beach Resort
12,Hobart,AU,80.01,-42.88,147.33,few clouds,St Ives Apartments
13,Airai,TL,83.41,-8.93,125.41,clear sky,Juvinal Martin
15,Carnarvon,AU,82.40,-24.87,113.63,clear sky,Hospitality Carnarvon
17,Yulara,AU,89.60,-25.24,130.99,clear sky,Desert Gardens Hotel - Ayers Rock Resort


In [10]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Weather} </dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [11]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
#Save the new DataFrame as a CSV file.
hotel_data_path = "weather_data_challenge/WeatherPy_vacation.csv"
hotel_df.to_csv(hotel_data_path)